In [1]:
from utilities.download import download_and_read_json_xz

# Example usage
url = "http://dadosabertos.c3sl.ufpr.br/curitibaurbs/2023_11_25_shapeLinha.json.xz"
df = download_and_read_json_xz(url)

In [2]:
from utilities.coordinates import format_coord

df['coordinates'] = df.apply(lambda row: format_coord(row['LAT'], row['LON']), axis=1)
df.drop(['LAT', 'LON'], axis=1, inplace=True)

In [3]:
df

,SHP,COD,DATE,coordinates
0,2612,602,2023-11-25,"(-49.29564508098565, -25.51231568263223)"
1,2612,602,2023-11-25,"(-49.29563592791851, -25.51259077193037)"
2,2612,602,2023-11-25,"(-49.29563168666107, -25.512727106234163)"
3,2612,602,2023-11-25,"(-49.29562744540368, -25.512859204361412)"
4,2612,602,2023-11-25,"(-49.2956182923366, -25.513149725321043)"
...,...,...,...,...
248791,4720,X50,2023-11-25,"(-49.30822725263312, -25.436443131455214)"
248792,4720,X50,2023-11-25,"(-49.30809518523198, -25.436366860559378)"
248793,4720,X50,2023-11-25,"(-49.3080087153841, -25.4362566788145)"
248794,4720,X50,2023-11-25,"(-49.307813584678264, -25.436163423917385)"


In [4]:
import pandas as pd
from geopy.distance import geodesic

def calculate_distances(group):
    group = group.reset_index()

    previous_shifted = group['coordinates'].shift(1)
    next_shifted = group['coordinates'].shift(-1)

    group['distance_to_prev'] = [geodesic(c1, c2).m if not pd.isna(c2) else 0 for c1, c2 in zip(group['coordinates'], previous_shifted)]

    group['distance_to_next'] = [geodesic(c1, c2).m if not pd.isna(c2) else 0 for c1, c2 in zip(group['coordinates'], next_shifted)]

    group = group.drop(columns='index')
    return group

# Group by SHP, COD, and DATE, then apply the distance calculation
df = df.groupby(['SHP', 'COD', 'DATE'], group_keys=False).apply(calculate_distances)

/tmp/ipykernel_1821119/3913659975.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['SHP', 'COD', 'DATE'], group_keys=False).apply(calculate_distances)


In [5]:
mean_distance_prev = df['distance_to_prev'].mean()
std_distance_prev = df['distance_to_prev'].std()
mean_distance_next = df['distance_to_next'].mean()
std_distance_next = df['distance_to_next'].std()


threshold_prev = mean_distance_prev + 3 * std_distance_prev
threshold_next = mean_distance_next + 3 * std_distance_next

df_filtered = df[(df['distance_to_prev'] <= threshold_prev) & (df['distance_to_next'] <= threshold_next)]


In [6]:
df_filtered = df_filtered.groupby(['SHP', 'COD', 'DATE'], group_keys=False).apply(calculate_distances)

df_filtered

/tmp/ipykernel_1821119/2271484324.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filtered = df_filtered.groupby(['SHP', 'COD', 'DATE'], group_keys=False).apply(calculate_distances)


,SHP,COD,DATE,coordinates,distance_to_prev,distance_to_next
0,1708,010,2023-11-25,"(-49.268802216214, -25.416432732579345)",0.000000,22.161966
1,1708,010,2023-11-25,"(-49.26860454212067, -25.41647121277954)",22.161966,20.191416
2,1708,010,2023-11-25,"(-49.26842497293825, -25.416512115585526)",20.191416,4.428712
3,1708,010,2023-11-25,"(-49.26842097976231, -25.416572661387978)",4.428712,5.292711
4,1708,010,2023-11-25,"(-49.26838868014249, -25.416626070744645)",5.292711,4.754216
...,...,...,...,...,...,...
148,5071,615,2023-11-25,"(-49.2922926532725, -25.476659902061197)",4.557484,7.369490
149,5071,615,2023-11-25,"(-49.292231020343735, -25.476622696473505)",7.369490,2.139475
150,5071,615,2023-11-25,"(-49.29221330858759, -25.476611218189845)",2.139475,12.905262
151,5071,615,2023-11-25,"(-49.29225817089336, -25.476447603807276)",12.905262,12.908449


In [7]:
df_filtered.drop(['distance_to_prev', 'distance_to_next'], axis=1, inplace=True)

In [8]:
from utilities.elastic import get_elastic_client

es = get_elastic_client()

/home/mateus/miniconda3/envs/data/lib/python3.9/site-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://elastic.tccurbstads.com:80' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [9]:
# join the cordinates into a list
df_filtered = df_filtered.groupby(['SHP', 'COD', 'DATE'], group_keys=False).agg({'coordinates': lambda x: list(x)})

In [10]:
df_filtered['coordinates'] = df_filtered['coordinates'].apply(lambda x: {'type': 'LineString', 'coordinates': x})

In [11]:
index_name = 'shape_linha'

index_settings = {
  "settings": {
    "number_of_shards": 1,
    "number_of_replicas": 1
  },
  "mappings": {
    "properties": {
      "SHP": {
        "type": "integer" 
      },
      "COD": { 
        "type": "keyword" 
      },
      "DATE": { 
        "type": "date", 
        "format": "yyyy-MM-dd"
      },
      "coordinates": { 
        "type": "geo_shape"
      }
    }
  }
}

if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=index_settings)

TlsError: TLS error caused by: TlsError(TLS error caused by: SSLError([SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1129)))

In [ ]:
df_filtered

,,,coordinates
SHP,COD,DATE,
1708,010,2023-11-25,"{'type': 'LineString', 'coordinates': [(-49.26..."
1709,011,2023-11-25,"{'type': 'LineString', 'coordinates': [(-49.26..."
1713,023,2023-11-25,"{'type': 'LineString', 'coordinates': [(-49.25..."
1714,024,2023-11-25,"{'type': 'LineString', 'coordinates': [(-49.29..."
1717,040,2023-11-25,"{'type': 'LineString', 'coordinates': [(-49.33..."
...,...,...,...
5064,779,2023-11-25,"{'type': 'LineString', 'coordinates': [(-49.30..."
5067,779,2023-11-25,"{'type': 'LineString', 'coordinates': [(-49.30..."
5068,722,2023-11-25,"{'type': 'LineString', 'coordinates': [(-49.35..."


In [12]:
from utilities.elastic import bulk_insert

bulk_insert(es, df_filtered, index_name)

TlsError: TLS error caused by: TlsError(TLS error caused by: SSLError([SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1129)))